# JSON

This page considers usage options of JSON format.

Find out more in article [JSON functions and operators](https://www.postgresql.org/docs/current/functions-json.html) in official postgres documentation.

In [1]:
docker run --rm -d\
    -e POSTGRES_PASSWORD=docker_app \
    --name json_example_pg \
    postgres:15.4 &> /dev/null
sleep 5

**Note:** Don't forget to stop the containers after all.

In [ ]:
docker stop json_example_pg

## Extracting fields

You can extract the value under a particular field of JSON by using the `-> '<field>'` syntax.

---

The following cell shows extracting a value under a specific JSON key.

In [6]:
docker exec -i json_example_pg psql -U postgres << EOF
SELECT ('{"a": 10, "b": 45}'::JSONB) -> 'a' ans; 
EOF

 ans 
-----
 10
(1 row)



The next example demonstrates how this works for columns in a table using the `->` operator combination.

In [ ]:
docker exec -i json_example_pg psql -U postgres << EOF
SELECT
    col->'a' AS a_value,
    col->'b' AS b_value,
    col->'b'->'y' b_y_value
FROM (
    VALUES
    ('{"a": 10, "b": {"x": 10, "y": 20}}'::JSONB),
    ('{"a": 15, "b": {"x": 1, "y": 45}}'::JSONB)
) AS temp(col);
EOF

 a_value |      b_value       | b_y_value 
---------+--------------------+-----------
 10      | {"x": 10, "y": 20} | 20
 15      | {"x": 1, "y": 45}  | 45
(2 rows)



## Changing value

You can change a `json` value by applying the `jsonb_set` function to it. You need to use the syntax `jsonb_set(value, '{field1, field2, ...}', <new_value>)`, which will return JSON where the value under the `'{field1, field2, ...}'` sequence of keys is changed to `<new_value>`.

---

The following cell applies `jsonb_set` to some JSON data. As a result, it returns JSON with the updated value:

In [35]:
docker exec -i json_example_pg psql -U postgres << EOF
SELECT jsonb_set('{"A": 80}'::JSONB, '{A}', '10');
EOF

 jsonb_set 
-----------
 {"A": 10}
(1 row)



## Condition on field

With syntax `<column name> & <value>` you can get boolean value that marks if json contains `<value>` key.

---

The following cell creates tables with different structure jsons and shows the application of the `?` operator to them.

In [7]:
docker exec -i json_example_pg psql -U postgres << EOF
SELECT
    col,
    col ? 'a' AS contains_a,
    col ? 'b' AS contains_b
FROM (
    VALUES
    ('{"a": 10}'::JSONB),
    ('{"b": 15}'::JSONB)
) AS temp(col);
EOF

    col    | contains_a | contains_b 
-----------+------------+------------
 {"a": 10} | t          | f
 {"b": 15} | f          | t
(2 rows)

